# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas 
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\ccana\Downloads\Starter_Code (15)\Starter_Code\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,touros,-5.1989,-35.4608,299.36,88,99,2.87,BR,1712881145
1,1,valparaiso,-33.0393,-71.6273,289.82,61,75,0.00,CL,1712881065
2,2,isafjordur,66.0755,-23.1240,270.10,94,100,12.47,IS,1712881146
3,3,bethel,41.3712,-73.4140,288.91,90,100,2.57,US,1712881044
4,4,fraile pintado,-23.9408,-64.7994,291.36,87,100,1.00,AR,1712881147


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display   

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
"Lat",  
color = "City", 
tiles = 'OSM',
#geo = True #  Commenting out because this line throws ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 85) & (city_data_df["Max Temp"] > 65)]
weather_filtered_df = city_data_df.loc[city_data_df['Wind Speed'] < 10]
weather_filtered_df = city_data_df.loc[city_data_df['Cloudiness'] < 5]
# Drop any rows with null values
# YOUR CODE HERE
weather_filtered_df.dropna()
# Display sample data
# YOUR CODE HERE
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,plettenberg bay,-34.0527,23.3716,284.32,77,0,1.60,ZA,1712881147
10,10,san patricio,28.0170,-97.5169,301.20,29,0,4.52,US,1712881149
18,18,ushuaia,-54.8000,-68.3000,280.96,71,0,12.35,AR,1712880881
21,21,al jawf,29.5000,38.7500,291.58,36,0,2.85,SA,1712881154
31,31,prince rupert,54.3161,-130.3201,280.20,63,0,5.14,CA,1712881158
...,...,...,...,...,...,...,...,...,...,...
522,522,queenstown,-31.8976,26.8753,277.33,87,0,0.64,ZA,1712881404
523,523,brisas de zicatela,15.8369,-97.0419,301.44,89,0,3.60,MX,1712881404
525,525,medina,24.4686,39.6142,295.39,18,0,0.00,SA,1712881405
527,527,zaragoza,41.6561,-0.8773,288.72,53,0,2.57,ES,1712881406


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = weather_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,plettenberg bay,ZA,-34.0527,23.3716,77,
10,san patricio,US,28.0170,-97.5169,29,
18,ushuaia,AR,-54.8000,-68.3000,71,
21,al jawf,SA,29.5000,38.7500,36,
31,prince rupert,CA,54.3161,-130.3201,63,
...,...,...,...,...,...,...
522,queenstown,ZA,-31.8976,26.8753,87,
523,brisas de zicatela,MX,15.8369,-97.0419,89,
525,medina,SA,24.4686,39.6142,18,
527,zaragoza,ES,41.6561,-0.8773,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000 
params = {'categories': 'accomodation.hotel',
        'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
plettenberg bay - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
prince rupert - nearest hotel: No hotel found
safford - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
manhush - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
diyarbakir - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
praia - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
estevan - nearest hotel: No hotel found
chubbuck - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
5,plettenberg bay,ZA,-34.0527,23.3716,77,No hotel found
10,san patricio,US,28.0170,-97.5169,29,No hotel found
18,ushuaia,AR,-54.8000,-68.3000,71,No hotel found
21,al jawf,SA,29.5000,38.7500,36,No hotel found
31,prince rupert,CA,54.3161,-130.3201,63,No hotel found
...,...,...,...,...,...,...
522,queenstown,ZA,-31.8976,26.8753,87,No hotel found
523,brisas de zicatela,MX,15.8369,-97.0419,89,No hotel found
525,medina,SA,24.4686,39.6142,18,No hotel found
527,zaragoza,ES,41.6561,-0.8773,53,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   color = "City",
                                   #geo = True Commenting out because this line throws ImportError: Geographic projection support requires: cartopy, geoviews, pyproj 
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
# YOUR CODE HERE

hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)